# Set up parameters that will be inherited to plot background topo maps, on top of which other maps will be created

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from obspy import read, read_inventory

from flovopy.asl.grid import Grid
from flovopy.asl.wrappers2 import find_event_files
from flovopy.core.mvo import dome_location

# -------------------------- Config --------------------------
PROJECTDIR      = "/Users/GlennThompson/Dropbox/BRIEFCASE/SSADenver"
LOCALPROJECTDIR = "/Users/GlennThompson/work/PROJECTS/SSADenver_local"
OUTPUT_DIR      = f"{LOCALPROJECTDIR}/asl_results"
INPUT_DIR       = f"{PROJECTDIR}/ASL_inputs/biggest_pdc_events"
GLOBAL_CACHE    = f"{PROJECTDIR}/asl_global_cache"

INVENTORY_XML   = "/Users/glennthompson/Dropbox/MV_Seismic_and_GPS_stations.xml"
DEM_DEFAULT     = Path("/Users/glennthompson/Dropbox/MONTSERRAT_DEM_WGS84_MASTER.tif")
GRIDFILE_DEFAULT= "/Users/glennthompson/Dropbox/MASTER_GRID_MONTSERRAT.pkl"

SMOOTH_SECONDS  = 1.0
MAX_LAG_SECONDS = 8.0
MIN_XCORR       = 0.5
best_file_nums  = [35, 36, 40, 52, 82, 83, 84, 116, 310, 338]

INV     = read_inventory(INVENTORY_XML)
gridobj = Grid.load(GRIDFILE_DEFAULT)
input_dir = Path(INPUT_DIR)
event_files = list(find_event_files(input_dir))
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print("Dome (assumed source) =", dome_location)

from importlib import reload
import flovopy.processing.envelopes as env
import flovopy.asl.envelope_locate as envloc
reload(env)
reload(envloc)
from flovopy.processing.envelopes import (
    align_waveforms,
    align_waveforms_global,
    envelopes_stream,
    envelope_delays,
    locate_with_grid_from_delays,
)
from flovopy.asl.envelope_locate import process_event, summarize_suite, parse_lags_column, \
    compute_pairwise_diffs, summarize_pairwise, estimate_speed_from_stable_pairs, plot_speed_vs_distance

# -------------------------- Run suite --------------------------
rows = []
per_event_lags = []  # store detailed lags to print later

for file_num in best_file_nums:
    try:
        event_path = str(event_files[file_num])
    except Exception as e:
        print(f"[WARN] file_num={file_num} not found in event_files list: {e}")
        continue

    eventdir       = Path(OUTPUT_DIR) / str(file_num)
    score_vs_c_png = str(eventdir / "score_vs_c.png")
    ccf_plot_dir   = str(eventdir / "ccf_plots")
    topo_png       = str(eventdir / "best_location_topo.png")
    eventdir.mkdir(parents=True, exist_ok=True)

    print(f"\n=== Event {file_num} ===")
    try:
        st = read(event_path).select(component="Z")
    except Exception as e:
        print(f"[ERROR] read failed for {event_path}: {e}")
        continue
    st.merge(fill_value="interpolate")
    st.detrend("linear")

    # Per-event
    res = process_event(st, gridobj, INV, GLOBAL_CACHE, dome_location, smooth_s=1.0, max_lag_s=8.0,
                min_corr=0.5, c_range=(0.1, 5.0), event_idx=file_num,
                output_dir=OUTPUT_DIR,           # <-- needed for score_vs_c.png
                topo_dem_path=DEM_DEFAULT,
           )       # optional, keeps topo plot
        # keep your Stage-A knobs here if you want to tweak:
        # min_delta_d_km=0.5, min_abs_lag_s=0.15, delta_d_weight=True, c_phys=(0.5,3.5)
    print(res)
    print(res["locator_result"]["speed"])
    rows.append(res["summary_row"])  

# ---------------- Suite summary -> stable pairwise -> speed & plot ----------------

# 1) Build the suite summary DataFrame and save it
df = summarize_suite(rows, f"{OUTPUT_DIR}/suite_summary.csv")
print("suite_summary columns:", list(df.columns))
print(df)

# 2) Parse per-event lags (prefer the global set; fall back if needed)
lags_col = (
    "lags_global" if "lags_global" in df.columns else
    ("lags_glb" if "lags_glb" in df.columns else None)
)
if lags_col is None:
    raise KeyError("No lags column found in suite summary (expected 'lags_global' or 'lags_glb').")

lags_glb = parse_lags_column(df[lags_col])          # -> list[dict{STA: lag_s}]
pairdiffs = compute_pairwise_diffs(lags_glb)        # -> list[dict{(STA,STA): dτ}]
stable_glb = summarize_pairwise(pairdiffs, "global")# -> DataFrame with sta_a/sta_b, mean_s, median_s, std, mad, ...

# Save the stable pairs (nice to keep for reproducibility)
stable_csv = Path(OUTPUT_DIR) / "pairwise_lagdiff_stats_global_stable.csv"
stable_glb.to_csv(stable_csv, index=False)
print(f"[stable] wrote -> {stable_csv}  (rows={len(stable_glb)})")

if stable_glb.empty:
    raise ValueError("Stable global pairwise table is empty — check your lags parsing and MAD filter.")

# 3) Estimate speed directly from the DataFrame (no need to re-read CSV)
res = estimate_speed_from_stable_pairs(
    grid=gridobj,
    inventory=INV,
    cache_dir=GLOBAL_CACHE,
    dome_location=dome_location,
    stable_df=stable_glb,         # <— pass the DataFrame
    use_value="mean_s",           # or "median_s"
    weight_with="mad_scaled",     # or "std" or None
    min_delta_d_km=0.0,           # relax if you want to keep all geometry
    min_abs_tau_s=0.0,
    c_bounds=(0.2, 7.0),
    uncert_floor_s=0.05,
)

print(f"Speed ≈ {res['speed_km_s']:.3f} km/s  (68% CI {res['ci_68'][0]:.3f}–{res['ci_68'][1]:.3f}) "
      f"using {res['n_pairs_used']} pairs")

per_pair = res["per_pair"]   # -> has delta_d_km, c_pair_km_s, weight

print(f"Speed (robust): {res['speed_km_s']:.3f} km/s "
      f"(68% CI {res['ci_68'][0]:.3f}-{res['ci_68'][1]:.3f}), "
      f"pairs used: {res['n_pairs_used']}")

# 4) Plot apparent speed vs Δd with a straight-line fit (intercept fixed at 0.5 km/s)
plot_speed_vs_distance(res, intercept=0.5, cmax=7.0)   # accepts dict OR res["per_pair"]

In [ ]:
print(df)
def _describe(series, name):
    s = pd.to_numeric(series, errors="coerce").dropna()
    if s.empty:
        return f"{name}: no finite values"
    return (f"{name}: n={len(s)}, mean={s.mean():.3f}, median={s.median():.3f}, "
            f"std={s.std(ddof=1):.3f}, min={s.min():.3f}, max={s.max():.3f}")

print("\n=== Descriptive stats ===")
print(_describe(df["c_at_dome_kms"], "c @ dome (km/s)"))
print(_describe(df["c_at_bestnode_kms"], "c @ best node (km/s)"))
#print(_describe(df["dome_to_best_km"], "distance: dome→best (km)"))
print(_describe(df["score_bestnode"], "best-node score"))
print(_describe(df["n_pairs_used"], "n_pairs used"))

# Print best time-lags (top |lag|) for each event for both methods
print("\n=== Top lags by |lag| per event ===")
for rec in per_event_lags:
    e = rec["event_idx"]
    ref_top = ", ".join([f"{sta}:{lag:+.2f}s" for sta, lag in rec["top_ref_lags"]])
    glb_top = ", ".join([f"{sta}:{lag:+.2f}s" for sta, lag in rec["top_global_lags"]])
    print(f"[{e}] ref: {ref_top}")
    print(f"[{e}] glb: {glb_top}")

In [ ]:
# Load the "stable pairs" CSV produced earlier
stable_csv = f"{OUTPUT_DIR}/pairwise_lagdiff_stats_global_stable.csv"
stab = pd.read_csv(stable_csv)

# Build per-pair tables once for mean_s and once for median_s
pp_mean = envloc.build_per_pair_from_stable(
    grid=gridobj, inventory=INV, cache_dir=GLOBAL_CACHE, dome_location=dome_location,
    stable_df=stab, use_value="mean_s", c_bounds=(0.2, 7.0)
)
pp_median = envloc.build_per_pair_from_stable(
    grid=gridobj, inventory=INV, cache_dir=GLOBAL_CACHE, dome_location=dome_location,
    stable_df=stab, use_value="median_s", c_bounds=(0.2, 7.0)
)

# Plot all 6 weighted fits with fixed intercept b=0.5 km/s
envloc.plot_six_weighted_fits(pp_mean, pp_median, intercept=None, cmax=7.0)

In [ ]:
stable_csv = f"{OUTPUT_DIR}/pairwise_lagdiff_stats_global_stable.csv"
stab = pd.read_csv(stable_csv)

# Build per-pair tables once for mean_s and once for median_s
df_mean = envloc.make_pair_geometry_from_stable(
    grid=gridobj, inventory=INV, cache_dir=GLOBAL_CACHE, dome_location=dome_location,
    stable_df=stab, use_value="mean_s", c_bounds=(0.2, 7.0)
)
df_median = envloc.make_pair_geometry_from_stable(
    grid=gridobj, inventory=INV, cache_dir=GLOBAL_CACHE, dome_location=dome_location,
    stable_df=stab, use_value="median_s", c_bounds=(0.2, 7.0)
)

# 1) Δt vs Δd with 6 WLS lines (free intercept in τ-space)
envloc.plot_delta_t_vs_delta_d_with_wls(df_mean, df_median, fixed_intercept_tau=None)

# (optional) force τ-intercept to 0 s to visualize bias sensitivity
# plot_delta_t_vs_delta_d_with_wls(df_mean, df_median, fixed_intercept_tau=0.0)

# 2) c_pair vs d_avg with 6 WLS lines (free intercept in c-space)
envloc.plot_cpair_vs_davg_with_wls(df_mean, df_median, fixed_intercept_c=None, cmax=7.0)

# (optional) fix shallow intercept if you want to compare to a hypothesized near-surface speed
# plot_cpair_vs_davg_with_wls(df_mean, df_median, fixed_intercept_c=0.5, cmax=7.0)

In [ ]:
# --- Apparent speed vs average radial distance, colored by Δd ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from flovopy.asl.distances import compute_or_load_distances

# Inputs you already have in your notebook:
# gridobj, INV, GLOBAL_CACHE (or CACHE_DIR), dome_location, OUTPUT_DIR
stable_csv = Path(OUTPUT_DIR) / "pairwise_lagdiff_stats_global_stable.csv"

# 1) Load stable pair table
stab = pd.read_csv(stable_csv)

# Ensure we have sta_a/sta_b (derive from 'pair' if needed) and clean codes
def _clean_sta(x: str) -> str:
    import re
    return re.sub(r"[^A-Za-z0-9]", "", str(x)).upper()

if not {"sta_a", "sta_b"}.issubset(stab.columns):
    if "pair" not in stab.columns:
        raise ValueError("Need 'sta_a'/'sta_b' or a 'pair' column.")
    ab = stab["pair"].astype(str).str.split("-", n=1, expand=True)
    ab.columns = ["sta_a", "sta_b"]
    stab = pd.concat([stab, ab], axis=1)

stab["sta_a"] = stab["sta_a"].apply(_clean_sta)
stab["sta_b"] = stab["sta_b"].apply(_clean_sta)

# 2) Distances from dome to each station (collapse to one vector per station)
node_dists, coords, meta = compute_or_load_distances(
    gridobj,
    cache_dir=(GLOBAL_CACHE if 'GLOBAL_CACHE' in globals() else CACHE_DIR),
    inventory=INV, stream=None, use_elevation=True
)

def _to_sta_key(seed_or_sta: str) -> str:
    parts = str(seed_or_sta).split(".")
    return parts[1] if len(parts) >= 2 else str(seed_or_sta)

dist_by_sta = {}
for full_id, vec in node_dists.items():
    key = _to_sta_key(full_id)
    dist_by_sta.setdefault(key, np.asarray(vec, float))  # first channel wins

# Dome node index
def _resolve_dome_idx(grid, dome):
    if isinstance(dome, int):
        return int(dome)
    glon = np.asarray(grid.gridlon).ravel()
    glat = np.asarray(grid.gridlat).ravel()
    if isinstance(dome, dict) and {"lon","lat"} <= set(dome):
        lon, lat = float(dome["lon"]), float(dome["lat"])
    elif isinstance(dome, (tuple, list)) and len(dome) == 2:
        lon, lat = float(dome[0]), float(dome[1])
    else:
        raise ValueError("dome_location must be node index, dict{'lon','lat'}, or (lon,lat).")
    return int(np.argmin((glon - lon)**2 + (glat - lat)**2))

dome_idx = _resolve_dome_idx(gridobj, dome_location)

# 3) Build per-pair rows for a chosen lag column
def _per_pair(df: pd.DataFrame, use_value: str) -> pd.DataFrame:
    rows = []
    for _, r in df.iterrows():
        a, b = r["sta_a"], r["sta_b"]
        if a not in dist_by_sta or b not in dist_by_sta:
            continue
        da_vec, db_vec = dist_by_sta[a], dist_by_sta[b]
        if not (np.isfinite(da_vec[dome_idx]) and np.isfinite(db_vec[dome_idx])):
            continue

        d_a = float(da_vec[dome_idx])
        d_b = float(db_vec[dome_idx])
        delta_d = abs(d_a - d_b)                 # km, separation wrt dome
        d_avg   = 0.5 * (d_a + d_b)              # km, average radial distance

        if use_value not in r or not np.isfinite(r[use_value]):
            continue
        tau = float(r[use_value])
        if tau == 0:
            continue

        c = delta_d / abs(tau)                   # km/s, apparent speed
        rows.append(dict(
            pair=f"{a}-{b}", sta_a=a, sta_b=b,
            delta_d_km=delta_d, d_avg_km=d_avg,
            delta_tau_s=abs(tau), c_pair_km_s=c
        ))
    return pd.DataFrame(rows)

pp_mean   = _per_pair(stab, "mean_s"   if "mean_s"   in stab.columns else "mean")
pp_median = _per_pair(stab, "median_s" if "median_s" in stab.columns else "median")

# 4) Filter to physical speeds (<= 7 km/s) and finite values
C_MAX = 7.0
def _phys(df):
    m = np.isfinite(df["c_pair_km_s"]) & np.isfinite(df["d_avg_km"]) & np.isfinite(df["delta_d_km"]) & (df["c_pair_km_s"] <= C_MAX)
    return df.loc[m].copy()

pp_mean   = _phys(pp_mean)
pp_median = _phys(pp_median)

# 5) Plot: apparent speed vs average radial distance, colored by Δd
fig, ax = plt.subplots(figsize=(8, 5))

sc1 = ax.scatter(pp_mean["d_avg_km"], pp_mean["c_pair_km_s"],
                 c=pp_mean["delta_d_km"], marker="o", s=40, alpha=0.85,
                 label=f"mean_s (n={len(pp_mean)})")
sc2 = ax.scatter(pp_median["d_avg_km"], pp_median["c_pair_km_s"],
                 c=pp_median["delta_d_km"], marker="^", s=46, alpha=0.85,
                 label=f"median_s (n={len(pp_median)})")

cb = plt.colorbar(sc1, ax=ax, pad=0.02)
cb.set_label("Δd between stations (km)")

ax.set_xlabel("Average radial distance of station pair, d_avg (km)")
ax.set_ylabel("Apparent speed c_pair = Δd / |Δτ| (km/s)")
ax.set_title("Apparent speed vs average radial distance (colored by Δd)")
ax.grid(alpha=0.3)
ax.set_ylim(0, C_MAX)
ax.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
# --- Apparent speed vs distance of closest station, colored by Δd ---

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from flovopy.asl.distances import compute_or_load_distances

# Inputs already in your notebook:
# gridobj, INV, GLOBAL_CACHE (or CACHE_DIR), dome_location, OUTPUT_DIR
stable_csv = Path(OUTPUT_DIR) / "pairwise_lagdiff_stats_global_stable.csv"

stab = pd.read_csv(stable_csv)

# Ensure sta_a/sta_b exist and clean codes
def _clean_sta(x: str) -> str:
    import re
    return re.sub(r"[^A-Za-z0-9]", "", str(x)).upper()

if not {"sta_a", "sta_b"}.issubset(stab.columns):
    if "pair" not in stab.columns:
        raise ValueError("Need 'sta_a'/'sta_b' or a 'pair' column.")
    ab = stab["pair"].astype(str).str.split("-", n=1, expand=True)
    ab.columns = ["sta_a", "sta_b"]
    stab = pd.concat([stab, ab], axis=1)

stab["sta_a"] = stab["sta_a"].apply(_clean_sta)
stab["sta_b"] = stab["sta_b"].apply(_clean_sta)

# Distances from dome to each station
node_dists, coords, meta = compute_or_load_distances(
    gridobj,
    cache_dir=(GLOBAL_CACHE if 'GLOBAL_CACHE' in globals() else CACHE_DIR),
    inventory=INV, stream=None, use_elevation=True
)

def _to_sta_key(seed_or_sta: str) -> str:
    parts = str(seed_or_sta).split(".")
    return parts[1] if len(parts) >= 2 else str(seed_or_sta)

dist_by_sta = {}
for full_id, vec in node_dists.items():
    key = _to_sta_key(full_id)
    dist_by_sta.setdefault(key, np.asarray(vec, float))  # first channel wins

def _resolve_dome_idx(grid, dome):
    if isinstance(dome, int):
        return int(dome)
    glon = np.asarray(grid.gridlon).ravel()
    glat = np.asarray(grid.gridlat).ravel()
    if isinstance(dome, dict) and {"lon","lat"} <= set(dome):
        lon, lat = float(dome["lon"]), float(dome["lat"])
    elif isinstance(dome, (tuple, list)) and len(dome) == 2:
        lon, lat = float(dome[0]), float(dome[1])
    else:
        raise ValueError("dome_location must be node index, dict{'lon','lat'}, or (lon,lat).")
    return int(np.argmin((glon - lon)**2 + (glat - lat)**2))

dome_idx = _resolve_dome_idx(gridobj, dome_location)

# Build per-pair table, but use d_min_km = min(d_a, d_b)
def _per_pair(df: pd.DataFrame, use_value: str) -> pd.DataFrame:
    rows = []
    for _, r in df.iterrows():
        a, b = r["sta_a"], r["sta_b"]
        if a not in dist_by_sta or b not in dist_by_sta:
            continue
        da_vec, db_vec = dist_by_sta[a], dist_by_sta[b]
        if not (np.isfinite(da_vec[dome_idx]) and np.isfinite(db_vec[dome_idx])):
            continue

        d_a = float(da_vec[dome_idx])
        d_b = float(db_vec[dome_idx])
        delta_d = abs(d_a - d_b)              # km
        d_min   = min(d_a, d_b)               # <- distance of the closer station

        if use_value not in r or not np.isfinite(r[use_value]):
            continue
        tau = float(r[use_value])
        if tau == 0:
            continue

        c = delta_d / abs(tau)                # km/s
        rows.append(dict(
            pair=f"{a}-{b}", sta_a=a, sta_b=b,
            delta_d_km=delta_d, d_min_km=d_min,
            delta_tau_s=abs(tau), c_pair_km_s=c
        ))
    return pd.DataFrame(rows)

pp_mean   = _per_pair(stab, "mean_s"   if "mean_s"   in stab.columns else "mean")
pp_median = _per_pair(stab, "median_s" if "median_s" in stab.columns else "median")

# Filter to physical speeds
C_MAX = 7.0
def _phys(df):
    m = (np.isfinite(df["c_pair_km_s"]) &
         np.isfinite(df["d_min_km"]) &
         np.isfinite(df["delta_d_km"]) &
         (df["c_pair_km_s"] <= C_MAX))
    return df.loc[m].copy()

pp_mean   = _phys(pp_mean)
pp_median = _phys(pp_median)

# Plot: apparent speed vs distance of closest station, colored by Δd
fig, ax = plt.subplots(figsize=(8, 5))

sc1 = ax.scatter(pp_mean["d_min_km"], pp_mean["c_pair_km_s"],
                 c=pp_mean["delta_d_km"], marker="o", s=40, alpha=0.85,
                 label=f"mean_s (n={len(pp_mean)})")
sc2 = ax.scatter(pp_median["d_min_km"], pp_median["c_pair_km_s"],
                 c=pp_median["delta_d_km"], marker="^", s=46, alpha=0.85,
                 label=f"median_s (n={len(pp_median)})")

cb = plt.colorbar(sc1, ax=ax, pad=0.02)
cb.set_label("Δd between stations (km)")

ax.set_xlabel("Distance of closer station to dome, d_min (km)")
ax.set_ylabel("Apparent speed c_pair = Δd / |Δτ| (km/s)")
ax.set_title("Apparent speed vs distance of closest station (colored by Δd)")
ax.grid(alpha=0.3)
ax.set_ylim(0, C_MAX)
ax.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
# stable_glb is the DataFrame you already build (with sta_a, sta_b, mean_s, median_s, std, mad, n)
reload(envloc)
fits = envloc.run_six_and_plot(gridobj, INV, GLOBAL_CACHE, dome_location, stable_glb, cmax=6.0, v0_prior=(0.9, 0.3))

for f in fits:
    print(f"{f['label']}: v0={f['v0']:.3f} km/s, g={f['g']:.4f} s^-1  (n={f['n_pairs']})")

In [ ]:
import matplotlib.pyplot as plt
reload(envloc)
from flovopy.asl.envelope_locate import *

stable_csv = f"{OUTPUT_DIR}/pairwise_lagdiff_stats_global_stable.csv"
stab = pd.read_csv(stable_csv)

# Build once for mean_s and once for median_s (adds d_min_km & d_avg_km)
pp_mean   = build_per_pair_from_stable(grid=gridobj, inventory=INV, cache_dir=GLOBAL_CACHE,
                                       dome_location=dome_location, stable_df=stab, use_value="mean_s",
                                       c_bounds=(0.2, 7.0), verbose=True)
pp_median = build_per_pair_from_stable(grid=gridobj, inventory=INV, cache_dir=GLOBAL_CACHE,
                                       dome_location=dome_location, stable_df=stab, use_value="median_s",
                                       c_bounds=(0.2, 7.0), verbose=True)

# Choose depth proxy on x-axis: "d_min_km" (closest station) or "d_avg_km" (average distance)
x_col = "d_min_km"

fits = []
for pp, stat_name in [(pp_mean, "mean_s"), (pp_median, "median_s")]:
    for wmode in ["inv_std", "inv_mad", "inv_n"]:
        res = fit_v0_beta_from_pairs(pp, x_col=x_col, weight_mode=wmode,
                                     v0_prior=None, beta_bounds=(0.0, 0.10),
                                     intercept_bounds=(0.2, 3.5), verbose=True)
        fits.append((stat_name, wmode, res))

# Plot the points and all 6 lines
fig, ax = plt.subplots(figsize=(7.0, 4.6))
ax.scatter(pp_mean[x_col], pp_mean["c_pair_km_s"], s=28, alpha=0.4, label="pairs (mean_s)")
ax.scatter(pp_median[x_col], pp_median["c_pair_km_s"], s=28, alpha=0.4, label="pairs (median_s)", marker="^")

xx = np.linspace(0, max(pp_mean[x_col].max(), pp_median[x_col].max())*1.05, 200)
for stat_name, wmode, res in fits:
    v0, beta, r2 = res["v0"], res["beta"], res["r2"]
    lbl = f"{stat_name} · {wmode}: v0={v0:.2f}, β={beta:.3f}, R²={r2:.2f}"
    ax.plot(xx, v0 + beta*xx, lw=1.8, label=lbl)

ax.set_xlabel("Depth proxy (km) — choose d_min or d_avg")
ax.set_ylabel("Apparent speed c_pair (km/s)")
ax.set_ylim(0, 7.0)
ax.grid(alpha=0.3)
ax.legend(fontsize=8, loc="best")
plt.tight_layout()
plt.show()

In [ ]:
combos = [
    ("mean_s","inv_std"),
    ("mean_s","inv_mad"),
    ("mean_s","inv_n"),
    ("median_s","inv_std"),
    ("median_s","inv_mad"),
    ("median_s","inv_n"),
]
for use_val, wby in combos:
    out = envloc.fit_v0_beta_from_pairs(
        grid=gridobj, inventory=INV, cache_dir=GLOBAL_CACHE, dome_location=dome_location,
        stable_df=stable_glb, use_value=use_val, weight_by=wby,
        depth_proxy="avg",   # or "min"
        v0_prior=0.3, beta_prior=0.1,
    )
    print(f"{use_val} · {wby}: v0={out['v0_km_s']:.3f}, beta={out['beta_km_s_per_km']:.4f}, R²={out['r2']:.3f}, n={out['n']}")